# Imports

In [ ]:
import os
import sys
import re
import numpy as np
import pandas as pd
import json

In [ ]:
from datetime import datetime, timedelta
import hashlib

In [ ]:
sys.path.append('../')
sys.path.append('../src/')
sys.path.append('../spell/')

In [ ]:
import EncodeCommand
import DeepLearningAnomalyDetection2

In [ ]:
from normal_utilities import *
from silent_utilities import *
from silent_utilities_2 import *
from preprocess_utilities import *

# Creation of datasets

## Normal execution

In [ ]:
fp = '../../../../../temp_logs/20240418/login01.laurel.log'
df = read(fp)
df_expanded = df['message'].apply(parse_audit_log).apply(pd.Series)
df = pd.concat([df, df_expanded], axis=1)
df.head()
df = preprocess_laurel(df)
save(df, fp + '.csv')

In [ ]:
normal_dataset_fp = fp + '.csv'
new_path = '../cleaned_data/normal.csv'
save(df, new_path)

## LinPeas noisy execution on laurel

In [ ]:
# First type of log file:
# laurel_anomalous_new
fp = '../data/laurel_anomalous_new/audit.log'
fps = [fp]
fps += [fp + f'.{str(i)}' for i in range(1,8)]
print(fps)

In [ ]:
for fp in fps:
    df = read(fp)
    df = preprocess_laurel(df)
    save(df, fp + '.csv')

## LinPeas silent execution on laurel

In [ ]:
# First type of log file:
# laurel_anomalous_new
fp = '../data/silent/audit.log'
fps = [fp]
fps += [fp + f'.{str(i)}' for i in range(1,11)]
print(fps)

In [ ]:
for fp in fps:
    df = read(fp)
    df = preprocess_laurel(df)
    save(df, fp + '.csv')

In [ ]:
import os

def concatenate_files(input_files, output_file):
    with open(output_file, 'w') as outfile:
        for filename in input_files:
            with open(filename, 'r') as infile:
                # Read each input file and write its content to the output file
                outfile.write(infile.read())
                outfile.write("\n")  # Optional: Adds a newline between files

# List of 10 input files (full paths or filenames)
input_files = ['../data/silent/audit.log', '../data/silent/audit.log.1', '../data/silent/audit.log.2', '../data/silent/audit.log.3', '../data/silent/audit.log.4', '../data/silent/audit.log.5', '../data/silent/audit.log.6', '../data/silent/audit.log.7', '../data/silent/audit.log.8', '../data/silent/audit.log.9', '../data/silent/audit.log.10']
input_files = input_files[::-1]
# Output file where the concatenated content will be written
output_file = 'silent.log'

# Call the function to concatenate the files
concatenate_files(input_files, output_file)

print(f"All files have been concatenated into {output_file}")

In [ ]:
# merge the csv files
fp = '../data/silent.log'
df = read(fp)
df = preprocess_laurel(df)
save(df, fp + '.csv')

# Preprocessing

## Data Reading

In [ ]:
normal = pd.read_csv('../cleaned_data/normal.csv')
anomalous = pd.read_csv('../cleaned_data/anomalous0.csv')
silent = pd.read_csv('../cleaned_data/silent.log.csv')

In [ ]:
encoder = EncodeCommand.EncodeCommand()
normal_ = preprocess(normal,encoder)
anomalous_ = preprocess(anomalous, encoder)
silent_ = preprocess(silent, encoder)

# Experiments and Results

## Statistics and ML

### Clustering

#### Spell

#### Spell (Modified Edit Distance)

### Dimensionality Reduction

#### PCA

#### t-SNE

#### uMAP

### Anomaly Detection

#### Isolation Forest

## DL Time Independent

### Normal vs Anomalous (secure)

In [ ]:
# see summary 6 to 9

### Normal vs Anomalous

In [ ]:
d_an_det = DeepLearningAnomalyDetection2.DeepLearningAnomalyDetection()
true_anomalies, reconstructed_anomalies, fp_ = d_an_det.train_test_model(normal_, anomalous_, 'autoencoder', laurel=1, plots=[0,0,1,1])

### Normal vs Silent

In [ ]:
d_an_det = DeepLearningAnomalyDetection2.DeepLearningAnomalyDetection()
true_anomalies, reconstructed_anomalies, fp_ = d_an_det.train_test_model(normal_, silent_, 'autoencoder', laurel=1, plots=[0,0,1,1])

In [ ]:
# remove features
normal_small = normal_.drop(['pid', 'ppid'], axis=1)
silent_small = silent_.drop(['pid', 'ppid'], axis=1)

In [ ]:
d_an_det = DeepLearningAnomalyDetection2.DeepLearningAnomalyDetection()
true_anomalies, reconstructed_anomalies, fp_ = d_an_det.train_test_model(normal_small, silent_small, 'autoencoder', laurel=1, plots=[0,0,1,1])

## DL Time Dependent

In [ ]:
# to complete